In [1]:
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

np.set_printoptions(precision=20)

MAX_DISTANT = 8  # 无人机最多飞8km

In [2]:
# 处理列表函数，同V0.5
def process_list(my_list):
    count_dict = {}
    coordinate = []
    for i, item in enumerate(my_list):
        count_dict[i] = item[0]
        coordinate.append(item[1])
    return count_dict, coordinate


# 计算距离函数
def calculate_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

In [3]:
# 生成路径函数，返回路径，总距离，和巡检杆塔数
def generate_route(points, count_dict, i):
    remaining_points = set(range(len(points)))
    remaining_points.remove(0)  # 还没巡检的位置集合（去除起始位置）
    route = []
    current_point = 0
    distance_traveled = 0  # 单次巡检路径（不超过MAX_DISTANT）
    total_distance = 0  # 该机巢总路径，涉及到耗时，越小越好。
    number = 0  # 该机巢寻到的数量
    time = 0

    # 当还有剩余点位未能巡检
    while remaining_points and time < 24:

        # 找到最近点的序号，并计算当前位置到最近的点距离
        nearest_point = min(remaining_points, key=lambda x: calculate_distance(points[current_point], points[x]))
        distance_to_nearest = calculate_distance(points[current_point], points[nearest_point])
        # print(distance_to_nearest)

        # 最近的点到机巢距离
        distance_to_nest = calculate_distance(points[nearest_point], points[0])

        # 当已经行驶的距离+飞到最近点位的距离+能飞回巢穴的距离 <=  最大里程， 则： 还能继续飞
        if distance_traveled + distance_to_nearest + distance_to_nest <= MAX_DISTANT:
            route.append(count_dict[nearest_point])
            remaining_points.remove(nearest_point)
            time += (distance_to_nearest / 5)
            if time <= 24:
                number += 1

            current_point = nearest_point
            distance_traveled += distance_to_nearest
            total_distance += distance_to_nearest
        elif distance_to_nearest > MAX_DISTANT / 2:
            # print('太远了,无法完成')
            remaining_points.remove(nearest_point)
            # total_distance += 10000
        else:
            # 返回起点加油
            route.append('b\n')
            current_point = 0
            total_distance += distance_to_nest  # 归巢的路程
            total_distance += 5  # 充一个小时电，相当于飞5公里。
            time += (distance_to_nest / 5)  # 归巢的时间
            time += 1  # 充电的时间
            distance_traveled = 0

    # 没有剩余点位了，回巢。当前点位是最后一个
    total_distance += calculate_distance(points[current_point], points[0])

    # print(route)
    # print('该机巢总路径为：', total_distance)  # 需要让每一组的total_distance的最大值尽可能的小。

    # 添加回到起始点的路径。返回的是相对每一组的序号。需要使用列表转化为总表格的序号。
    return route, total_distance, number


# 整体路径生成，调用10次generate_route函数
def total_route_generator(dianwei_list):
    total_route = []
    longest = 0  #
    dian_num = 0
    for i in range(10):
        count_dict, coordinate = process_list(dianwei_list[i])
        # print(i + 1, '号机巢需要巡检的杆塔数量是', len(dianwei_list[i]))
        points = np.array(coordinate)
        route, cost, num = generate_route(points, count_dict, i)
        # print(cost)
        total_route.append(route)
        # total_route.append(count_dict)
        if longest < cost:
            longest = cost
        dian_num += num
    return total_route, longest, dian_num


# 推理函数（最终的搜索），进行search_number次聚类和航线生成，根据min_index和max_index即花费时间和巡检塔数量挑选最优解，进行保存。
def inference_model(location, search_number, output_jichao_path, output_hangxian_path):
    A, B, C, D = [], [], [], []
    for i in range(search_number):
        # print('开始搜寻------------------------------正在进行第',i+1, '轮。')
        kmeans = KMeans(n_clusters=10)  # 使用不同的随机初始状态
        kmeans.fit(location)  # 聚类
        jichao = kmeans.cluster_centers_  # 得到聚类中心坐标，即机巢位置
        indexed_cluster_centers = list(enumerate(jichao, start=1))  # 制作列表用于保存
        A.append(indexed_cluster_centers)  # 将此次生成的坐标保存至A列表，用于后期保存调用等。

        dianwei_list = [[[0, [x, y]]] for x, y in jichao.tolist()]  # 杆塔点位分组，得到dianwei_list，第一维度是10，第二维度是序号和坐标位置。
        for j, label in enumerate(kmeans.labels_):
            dianwei_list[label].append([int(j + 1), [location[j][0], location[j][1]]])

        total_route, longest, dian_num = total_route_generator(
            dianwei_list)  # 根据机巢坐标和点位列表生成总体路径（维度是[search_number,10,）。
        B.append(total_route)
        C.append(longest)
        D.append(dian_num)
        total_route, longest = 0, 0

        # print(A, '第------------------',i+1,'次聚类结果------')
        # print(B,'第------------------',i+1,'次结果------')
        # print(C,'第------------------',i+1,'次结果------')
        # print('完成第', i + 1, '轮搜寻！！！共巡检', dian_num, '个杆塔')

    min_index = C.index(min(C))  # 找到花费时间最少的一次的索引
    max_index = D.index(max(D))  # 找到杆塔最多的一次索引
    # print('搜索第',min_index+1,'次结果最佳，最长的巡检时间为',float(C[min_index])/5,'小时，保存此次结果！')
    # print('搜索第',max_index+1,'次结果最佳，巡到的数量为',float(D[max_index]),'个，保存此次结果！')

    # save_result(result1, result2)
    with open(output_jichao_path, 'w') as file:
        for index, center in A[max_index]:
            x, y = center
            file.write(f"{index} {x:.14f} {y:.14f}\n")
        # print('*********************************Finished jichao***************************************')

    with open(output_hangxian_path, 'w') as file:
        # print('你好',len(B[min_index]))
        for i in range(10):
            file.write(str(i + 1))
            for item in B[max_index][i]:
                # print(item)
                if item == 'b\n':
                    # print('BBBBBBB-------------------------')
                    file.write(' ' + 'b' + '\n' + str(i + 1))
                else:
                    file.write(' ' + str(item))
            file.write(' ' + 'b' + '\n')
    return

In [4]:
if __name__ == "__main__":
    # 设置输入图像路径和输出结果路径
    # input_ganta_path = "/home/bml/.storage/mnt/v-zhj9oak0e0ajwe8z/saiti1/ganta.xlsx"
    #### 请各团队将以下地址的XXX替换为自己团队的首字母缩写，即平台账号名 ####
    # output_jichao_path = "/home/bml/.storage/mnt/v-zhj9oak0e0ajwe8z/saiti1/result/hebeidky_jichao.txt"
    # output_hangxian_path = "/home/bml/.storage/mnt/v-zhj9oak0e0ajwe8z/saiti1/result/hebeidky_hangxian.txt"
    input_ganta_path = './Samples/杆塔数据样例2.xlsx'
    output_jichao_path = './Results/hebeidky_jichao.txt'
    output_hangxian_path = './Results/hebeidky_hangxian.txt'
    data = pd.read_excel(input_ganta_path, dtype=np.float64)
    X = data.iloc[:, 1:3].values

    # 记录开始时间
    start_time = time.time()
    search_number = 4
    # 模型推理
    inference_model(X, search_number, output_jichao_path, output_hangxian_path)

    # 记录结束时间
    end_time = time.time()

    # 输出总时间
    elapsed_time = end_time - start_time
    print(f"总共用时: {elapsed_time} 秒")


F:\miniconda\envs\torch_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
F:\miniconda\envs\torch_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
F:\miniconda\envs\torch_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
F:\miniconda\envs\torch_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set 

总共用时: 2.209667921066284 秒
